# Good-turing+Katz backoff

In [2]:
from math import log
from n_gram import *
import csv

In [3]:
# construct positive model
txt = "pos_pre.txt"
with open('%s' % txt,'r') as file:
     file_str = file.read()
file_str = file_str.replace('\n',' ')

length_pos,uni_word_pos,uni_cnt_pos,uni_prob_pos, \
bi_word_pos,bi_cnt_pos,bi_prob_pos,tri_word,tri_cnt_pos,tri_prob= gram(file_str)

# construct negative model
txt = "neg_pre.txt"
with open('%s' % txt,'r') as file:
     file_str = file.read()
file_str = file_str.replace('\n',' ')

length_neg,uni_word_neg,uni_cnt_neg,uni_prob_neg, \
bi_word_neg,bi_cnt_neg,bi_prob_neg,tri_word,tri_cnt_neg,tri_prob= gram(file_str)

In [4]:
"""
Funtion to calculate good_turing counts
"""
def Good_turing (k,uni_cnt,bi_cnt):
    
    # Good_turing for uni_word_cnt
    Nc_uni = {}
    for (key,value) in uni_cnt.items():
        if value in Nc_uni:
            Nc_uni[value]+=1
        else:
            Nc_uni[value]=1
    
    uni_cnt_good = {}
    uni_cnt_good['<UNK>']=float(Nc_uni[1])

    for (key,value) in uni_cnt.items():
        if value>k:
            uni_cnt_good[key]=value
        else:
            c = float(value)
            Nc = float(Nc_uni[c])
            N1 = float(Nc_uni[1])
            Nc1 = float(Nc_uni[c+1])
            Nk1 = float(Nc_uni[k+1])
            uni_cnt_good[key]=((c+1)*Nc1/Nc-c*(k+1)*Nk1/N1)/(1-(k+1)*Nk1/N1)
    
    # Good_turing for bi_word_cnt
    Nc_bi = {}
    for (key,value) in bi_cnt.items():
        if value in Nc_bi:
            Nc_bi[value]+=1
        else:
            Nc_bi[value]=1
    bi_cnt_good = {}
    for (key,value) in bi_cnt.items():
        if value>k:
            bi_cnt_good[key]=float(value)
        else:
            c = float(value)
            Nc = float(Nc_bi[c])
            N1 = float(Nc_bi[1])
            Nc1 = float(Nc_bi[c+1])
            Nk1 = float(Nc_bi[k+1])
            bi_cnt_good[key]=((c+1)*Nc1/Nc-c*(k+1)*Nk1/N1)/(1-(k+1)*Nk1/N1)
    return uni_cnt_good,bi_cnt_good
 

In [5]:
"""
Function to calculate backoff probability
"""
def calc_backoff(pre_word,word,length,uni_cnt,uni_cnt_good,bi_cnt,bi_cnt_good):
    tuple = (pre_word,word)
    if pre_word not in uni_cnt.keys():
        pre_word = '<UNK>'
    
    if tuple in bi_cnt:
        return log(float(bi_cnt_good[tuple])/uni_cnt_good[pre_word])
    else:
        total_bi_prob = 0.0
        for (key,value) in bi_cnt.items():
            if key[0] == pre_word:              
                total_bi_prob += float(bi_cnt_good[key])/uni_cnt[key[0]]
        alpha = 1-total_bi_prob
        if alpha == 0: return -float("inf")
        else: return log(alpha*float(uni_cnt_good[pre_word])/length)


        